# 라이브러리 불러오기

In [1]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1차 모델

In [3]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
fun_data = survey[['content_id', 'fun']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
fun_data['content_id'] = fun_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
fun_data = fun_data[fun_data['content_id'] != 0].reset_index().drop(columns=['index'])
fun_data['content_id'] = fun_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(fun_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
fun_data = pd.merge(fun_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
fun_data['fun'] = fun_data['fun'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

fun_data['content'] = fun_data['content'].apply(cleaned_content)

final_df = fun_data[['content', 'fun']]

# 라벨별 개수 확인
print(final_df['fun'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['fun'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['fun'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_59819/597480291.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fun_data['content_id'] = fun_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


fun
 0    4017
 1     939
-1      67
Name: count, dtype: int64


train_data fun
 0    3217
 1     749
-1      52
Name: count, dtype: int64

 test_data fun
 0    818
 1    184
-1      3
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [ ]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["fun"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['fun'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './fun_model/check_point/try_1',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train() 

In [ ]:
trainer.save_model("./fun_model/try_1/fun_model1")
tokenizer.save_pretrained("./fun_model/try_1/fun_tokenizer1")

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./fun_model/try_1/fun_model1"
tokenizer_path = "./fun_model/try_1/fun_tokenizer1"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 1001/1001 [00:11<00:00, 84.19it/s]


,content,fun,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0,0


In [6]:
print(train_data['fun'].value_counts())
print(test_data['fun'].value_counts())

fun
 0    3204
 1     747
-1      52
Name: count, dtype: int64
fun
 0    815
 1    183
-1      3
Name: count, dtype: int64


In [7]:
print(classification_report(test_data['fun'], test_data['pred']))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.98      0.92      0.95       815
           1       0.71      0.92      0.80       183

    accuracy                           0.92      1001
   macro avg       0.56      0.61      0.58      1001
weighted avg       0.93      0.92      0.92      1001



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [10]:
print(classification_report(test_data['fun'], test_data['pred']))
test_data2 = test_data[test_data['fun'] != 0]
print(classification_report(test_data2['fun'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.98      0.92      0.95       815
           1       0.71      0.92      0.80       183

    accuracy                           0.92      1001
   macro avg       0.56      0.61      0.58      1001
weighted avg       0.93      0.92      0.92      1001

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.00      0.00      0.00         0
           1       0.99      0.92      0.95       183

    accuracy                           0.91       186
   macro avg       0.33      0.31      0.32       186
weighted avg       0.97      0.91      0.94       186



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [12]:
test = ['재밌어요!', '존잼존잼', '무섭긴 한데 재밌었음!!', 
'노잼', '진짜 돈 아까움. 노잼 그 자체', '재미 하나도 없어요', '스토리도 없고 재미도 없고 인테리어도 없고',
'대존잼', '진짜진짜 재밌었어요', '스토리 그냥 저냥']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

문장: 재밌어요!, 평가: 2
---------- 

문장: 존잼존잼, 평가: 2
---------- 

문장: 무섭긴 한데 재밌었음!!, 평가: 2
---------- 

문장: 노잼, 평가: 2
---------- 

문장: 진짜 돈 아까움. 노잼 그 자체, 평가: 2
---------- 

문장: 재미 하나도 없어요, 평가: 2
---------- 

문장: 스토리도 없고 재미도 없고 인테리어도 없고, 평가: 2
---------- 

문장: 대존잼, 평가: 2
---------- 

문장: 진짜진짜 재밌었어요, 평가: 2
---------- 

문장: 스토리 그냥 저냥, 평가: 1
---------- 



## 1차 평가
- 긍정이랑 중립은 그냥저냥인데
- 부정 댓글...

# 2차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
fun_data = survey[['content_id', 'fun']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
fun_data['content_id'] = fun_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
fun_data = fun_data[fun_data['content_id'] != 0].reset_index().drop(columns=['index'])
fun_data['content_id'] = fun_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(fun_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
fun_data = pd.merge(fun_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
fun_data['fun'] = fun_data['fun'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

fun_data['content'] = fun_data['content'].apply(cleaned_content)

final_df = fun_data[['content', 'fun']]

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63567/3459739587.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fun_data['content_id'] = fun_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


In [3]:
# 데이터 추가
new_data = [
    ('존잼 그 잡채', 1), ('개재밌음. 개추.', 1), ('재미있는 테마 만났어요.', 1), ('대박 재미짜나!!!!', 1),
    ('즐겁고 신나고 싶다? 여기로 오세요', 1), ('존잼임', 1), ('재밌어요!! 즐거워요!!', 1), ('신나게 놀고 왔어요', 1),
    ('너무 즐거워!! 신나!!', 1), ('이런 테마라면 맨날 할 듯', 1), ('걍 시간이 순식간에 지나감', 1),
    ('대존잼', 1), ('특별한 건 없었는데, 그냥 즐거웠어요.', 1), ('솔직히 너무 낡았는데, 재밌어서 넘어감', 1),
    ('즐겁게 놀다 갑니다.', 1), ('재미없는 건 아니었어요.', 1),
    ('좋은 팀워크로 함께 퍼즐을 풀다 보니 존잼 그 자체', 1), ('방탈출의 매력에 푹 빠져 개재밌게 놀았어!', 1),
    ('독특한 테마와 함께해서 정말 즐겁고 재미있었어요.', 1), ('흥미진진한 상황에서 대박 재미를 느낄 수 있었어!', 1),
    ('쉽게 놀 수 있는데도 즐겁고 신나는 분위기에 푹 빠져버렸어요', 1), ('진짜 존잼! 다양한 퍼즐들이 기대 이상으로 재미를 더해줘', 1),
    ('팀원들과 함께 놀다 보니 정말 즐거워서 시간 가는 줄 몰랐어요!', 1), ('진심 즐겁고 신나서 계속 놀고 싶었던 곳이에요', 1),
    ('이 정도면 너무 즐거운 시간을 보낸 것 같아!', 1), ('흥미로운 테마로 계속해서 놀고 싶어졌어요. 정말 즐거웠어!', 1),
    ('개노잼', -1), ('씹노잼', -1), ('핵노잼', -1),
    ('돈 개아까움', -1), ('이런 걸 돈 주고 했다니.', -1), ('너무 재미 없었어요.', -1), ('오늘 이후로 방탈 안 할 듯', -1),
    ('개재미없었어여', -1), ('하고 나니 우울하기만 함', -1), ('재미도 감동도 없잖아?', -1), ('재미 없는 걸 재미 없다고 하지 또 뭐라고 함?', -1), 
    ('실화냐?', -1), ('재미없어서 어이 없던 건 처음이다', -1), ('없다 재미', -1), ('안 재미있잖아', -1),
    ('재미있는 건 아니었음', -1), ('심심하다', -1), ('돈 주고 이렇게 심심해도 됨?', -1), ('재미없어서 울어볼까?', -1),
    ('정성스럽게 잼없게한다', -1), ('대노잼', -1), ('후회만 가득', -1),
    ('너무 별로였어, 개노잼', -1), ('돈 주고 가서 정말 후회했어, 씹노잼', -1), ('퍼즐이 지루하고 재미가 없었어, 핵노잼', -1),
    ('돈이 아까웠어, 이런 걸로 돈 주고 하는 게 아니었는데', -1), ('이런 걸 돈 주고 했다는 게 너무 아까워서 기분 나쁘다', -1),
    ('재미 없었어, 다음에는 이런 거 하지 말아야지', -1), ('하고 나니까 정말 우울해졌어, 개재미없었어여', -1),
    ('재미와 감동이 전혀 없었어, 돈 주고 이런 걸 기대한 내가 멍청한 듯', -1), ('이런 걸 기대하고 들어갔다가 정말 실망했어, 재미 없는데 돈까지 들었잖아', -1),
    ('기대 이하로 심심해서 진짜 울고 싶어졌어, 정말 쓸데없는 경험이었어', -1)
]

new_df = pd.DataFrame(new_data, columns=['content', 'fun'])
final_df = pd.concat([final_df, new_df])

In [4]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

In [5]:
# 라벨별 개수 확인
print(final_df['fun'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['fun'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['fun'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

fun
 0    4017
 1     965
-1      99
Name: count, dtype: int64


train_data fun
 0    3217
 1     773
-1      75
Name: count, dtype: int64

 test_data fun
 0    778
 1    178
-1      2
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4065
중복 제거 전 테스트 데이터셋: 958




중복 제거 후 학습 데이터셋: 4050
중복 제거 후 테스트 데이터셋: 954


In [10]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["fun"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['fun'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './fun_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 50,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
# train 진행
trainer.train() 

trainer.save_model("./fun_model/try_2/fun_model2")
tokenizer.save_pretrained("./fun_model/try_2/fun_tokenizer2")




Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/12700 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_60030/713598392.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  4%|▍         | 500/12700 [01:29<35:50,  5.67it/s]

{'loss': 0.3404, 'learning_rate': 4.8031496062992124e-05, 'epoch': 1.97}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_60030/713598392.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  8%|▊         | 1000/12700 [02:59<35:05,  5.56it/s] 

{'loss': 0.2439, 'learning_rate': 4.606299212598425e-05, 'epoch': 3.94}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_60030/713598392.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 12%|█▏        | 1500/12700 [04:28<32:21,  5.77it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_60030/713598392.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 98%|█████████▊| 12500/12700 [36:56<00:47,  4.18it/s]

{'loss': 0.0031, 'learning_rate': 7.874015748031496e-07, 'epoch': 49.21}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_60030/713598392.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 12700/12700 [37:45<00:00,  5.61it/s]

{'train_runtime': 2265.3625, 'train_samples_per_second': 89.39, 'train_steps_per_second': 5.606, 'train_loss': 0.07451685796572467, 'epoch': 50.0}


('./fun_model/try_2/fun_tokenizer2/tokenizer_config.json',
 './fun_model/try_2/fun_tokenizer2/special_tokens_map.json',
 './fun_model/try_2/fun_tokenizer2/vocab.txt',
 './fun_model/try_2/fun_tokenizer2/added_tokens.json')

In [6]:
new_test_data = [
    ('정성스럽게 만들었는데 왜 이리 대노잼인지 이해가 안 돼', -1), ('돈 주고 갔는데 후회만 가득해서 너무 실망했어', -1),
    ('퍼즐이 너무 어려워서 즐기는 게 아니라 고생하는 느낌이었어', -1), ('이런 걸로 돈 주고 시간 낭비하는 게 너무 아까워', -1),
    ('하고 나니까 정말 즐거운 게 없어서 정말 심심하고 지루했어', -1)
]

new_test_data = pd.DataFrame(new_test_data, columns=['content', 'fun'])
test_data = pd.concat([test_data, new_test_data])

test_data['content'] = test_data['content'].apply(cleaned_content)
test_data['content'] = test_data['content'].apply(kiwi_clean)

In [7]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./fun_model/try_2/fun_model2"
tokenizer_path = "./fun_model/try_2/fun_tokenizer2"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 959/959 [00:13<00:00, 69.52it/s]


,content,fun,pred
0,스토리 좋다 문제 아쉽다 아직 많이 부족,0,0
1,인 장치 많다 무난 테마,0,0
2,재밌다 바보 열다 안 열다 복기 때 더 재밌다 신묘 테마 버전 걸리다,1,1
3,개인 공간 사용 부분 좋다 따다 두 문제 허 이다 가이드 별로 이다 생각 꽃길 너무...,0,0
4,테마 기준 걸리다,0,0
...,...,...,...
0,정성 만들다 왜 이리 대노잼 이다 이해 안 되다,-1,1
1,돈 주다 가다 후회 가득 너무 실망,-1,0
2,퍼즐 너무 어렵다 즐기다 아니다 고생 느낌 이다,-1,0
3,이런 돈 주다 시간 낭비 너무 아깝다,-1,-1


In [9]:
print('0포함', '\n', classification_report(test_data['fun'], test_data['pred']))
test_data2 = test_data[test_data['fun'] != 0]
print('0 제외', '\n', classification_report(test_data2['fun'], test_data2['pred']))

0포함 
               precision    recall  f1-score   support

          -1       0.18      0.43      0.25         7
           0       0.94      0.92      0.93       775
           1       0.74      0.77      0.76       177

    accuracy                           0.89       959
   macro avg       0.62      0.71      0.65       959
weighted avg       0.90      0.89      0.90       959

0 제외 
               precision    recall  f1-score   support

          -1       1.00      0.43      0.60         7
           0       0.00      0.00      0.00         0
           1       0.99      0.77      0.87       177

    accuracy                           0.76       184
   macro avg       0.66      0.40      0.49       184
weighted avg       0.99      0.76      0.86       184



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [10]:
test = ['재밌어요!', '존잼존잼', '무섭긴 한데 재밌었음!!', 
'노잼', '진짜 돈 아까움. 노잼 그 자체', '재미 하나도 없어요', '스토리도 없고 재미도 없고 인테리어도 없고',
'대존잼', '진짜진짜 재밌었어요', '스토리 그냥 저냥']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent} → 평가: {pred}')
    print("-"*30, '\n')

문장: 재밌어요! → 평가: 2
------------------------------ 

문장: 존잼존잼 → 평가: 2
------------------------------ 

문장: 무섭긴 한데 재밌었음!! → 평가: 1
------------------------------ 

문장: 노잼 → 평가: 0
------------------------------ 

문장: 진짜 돈 아까움. 노잼 그 자체 → 평가: 2
------------------------------ 

문장: 재미 하나도 없어요 → 평가: 0
------------------------------ 

문장: 스토리도 없고 재미도 없고 인테리어도 없고 → 평가: 0
------------------------------ 

문장: 대존잼 → 평가: 0
------------------------------ 

문장: 진짜진짜 재밌었어요 → 평가: 2
------------------------------ 

문장: 스토리 그냥 저냥 → 평가: 1
------------------------------ 



# 3차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
fun_data = survey[['content_id', 'fun']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
fun_data['content_id'] = fun_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
fun_data = fun_data[fun_data['content_id'] != 0].reset_index().drop(columns=['index'])
fun_data['content_id'] = fun_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(fun_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
fun_data = pd.merge(fun_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
fun_data['fun'] = fun_data['fun'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

fun_data['content'] = fun_data['content'].apply(cleaned_content)

final_df = fun_data[['content', 'fun']]

# 데이터 추가
new_data = [
    ('존잼 그 잡채', 1), ('개재밌음. 개추.', 1), ('재미있는 테마 만났어요.', 1), ('대박 재미짜나!!!!', 1),
    ('즐겁고 신나고 싶다? 여기로 오세요', 1), ('존잼임', 1), ('재밌어요!! 즐거워요!!', 1), ('신나게 놀고 왔어요', 1),
    ('너무 즐거워!! 신나!!', 1), ('이런 테마라면 맨날 할 듯', 1), ('걍 시간이 순식간에 지나감', 1),
    ('대존잼', 1), ('특별한 건 없었는데, 그냥 즐거웠어요.', 1), ('솔직히 너무 낡았는데, 재밌어서 넘어감', 1),
    ('즐겁게 놀다 갑니다.', 1), ('재미없는 건 아니었어요.', 1),
    ('좋은 팀워크로 함께 퍼즐을 풀다 보니 존잼 그 자체', 1), ('방탈출의 매력에 푹 빠져 개재밌게 놀았어!', 1),
    ('독특한 테마와 함께해서 정말 즐겁고 재미있었어요.', 1), ('흥미진진한 상황에서 대박 재미를 느낄 수 있었어!', 1),
    ('쉽게 놀 수 있는데도 즐겁고 신나는 분위기에 푹 빠져버렸어요', 1), ('진짜 존잼! 다양한 퍼즐들이 기대 이상으로 재미를 더해줘', 1),
    ('팀원들과 함께 놀다 보니 정말 즐거워서 시간 가는 줄 몰랐어요!', 1), ('진심 즐겁고 신나서 계속 놀고 싶었던 곳이에요', 1),
    ('이 정도면 너무 즐거운 시간을 보낸 것 같아!', 1), ('흥미로운 테마로 계속해서 놀고 싶어졌어요. 정말 즐거웠어!', 1),
    ('개노잼', -1), ('씹노잼', -1), ('핵노잼', -1),
    ('돈 개아까움', -1), ('이런 걸 돈 주고 했다니.', -1), ('너무 재미 없었어요.', -1), ('오늘 이후로 방탈 안 할 듯', -1),
    ('개재미없었어여', -1), ('하고 나니 우울하기만 함', -1), ('재미도 감동도 없잖아?', -1), ('재미 없는 걸 재미 없다고 하지 또 뭐라고 함?', -1), 
    ('실화냐?', -1), ('재미없어서 어이 없던 건 처음이다', -1), ('없다 재미', -1), ('안 재미있잖아', -1),
    ('재미있는 건 아니었음', -1), ('심심하다', -1), ('돈 주고 이렇게 심심해도 됨?', -1), ('재미없어서 울어볼까?', -1),
    ('정성스럽게 잼없게한다', -1), ('대노잼', -1), ('후회만 가득', -1),
    ('너무 별로였어, 개노잼', -1), ('돈 주고 가서 정말 후회했어, 씹노잼', -1), ('퍼즐이 지루하고 재미가 없었어, 핵노잼', -1),
    ('돈이 아까웠어, 이런 걸로 돈 주고 하는 게 아니었는데', -1), ('이런 걸 돈 주고 했다는 게 너무 아까워서 기분 나쁘다', -1),
    ('재미 없었어, 다음에는 이런 거 하지 말아야지', -1), ('하고 나니까 정말 우울해졌어, 개재미없었어여', -1),
    ('재미와 감동이 전혀 없었어, 돈 주고 이런 걸 기대한 내가 멍청한 듯', -1), ('이런 걸 기대하고 들어갔다가 정말 실망했어, 재미 없는데 돈까지 들었잖아', -1),
    ('기대 이하로 심심해서 진짜 울고 싶어졌어, 정말 쓸데없는 경험이었어', -1)
]

new_df = pd.DataFrame(new_data, columns=['content', 'fun'])
final_df = pd.concat([final_df, new_df])

from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

    # 라벨별 개수 확인
print(final_df['fun'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['fun'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['fun'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_65960/4163611660.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fun_data['content_id'] = fun_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


fun
 0    4017
 1     965
-1      99
Name: count, dtype: int64


train_data fun
 0    3217
 1     773
-1      75
Name: count, dtype: int64

 test_data fun
 0    778
 1    178
-1      2
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4065
중복 제거 전 테스트 데이터셋: 958




중복 제거 후 학습 데이터셋: 4050
중복 제거 후 테스트 데이터셋: 954


In [3]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["fun"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['fun'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './fun_model/check_point/try_3',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 30,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
# train 진행
trainer.train() 

trainer.save_model("./fun_model/try_3/fun_model3")
tokenizer.save_pretrained("./fun_model/try_3/fun_tokenizer3")




Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/7620 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63808/2816094592.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  7%|▋         | 500/7620 [02:04<29:23,  4.04it/s]

{'loss': 0.3484, 'learning_rate': 4.671916010498688e-05, 'epoch': 1.97}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63808/2816094592.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 13%|█▎        | 1000/7620 [04:08<26:54,  4.10it/s]

{'loss': 0.2539, 'learning_rate': 4.343832020997376e-05, 'epoch': 3.94}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63808/2816094592.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|█▉        | 1500/7620 [06:10<25:06,  4.06it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63808/2816094592.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 98%|█████████▊| 7500/7620 [30:39<00:29,  4.12it/s]

{'loss': 0.0269, 'learning_rate': 7.874015748031496e-07, 'epoch': 29.53}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63808/2816094592.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7620/7620 [31:08<00:00,  4.08it/s]

{'train_runtime': 1869.0213, 'train_samples_per_second': 65.007, 'train_steps_per_second': 4.077, 'train_loss': 0.12435305628563788, 'epoch': 30.0}


('./fun_model/try_3/fun_tokenizer3/tokenizer_config.json',
 './fun_model/try_3/fun_tokenizer3/special_tokens_map.json',
 './fun_model/try_3/fun_tokenizer3/vocab.txt',
 './fun_model/try_3/fun_tokenizer3/added_tokens.json')

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./fun_model/try_3/fun_model3"
tokenizer_path = "./fun_model/try_3/fun_tokenizer3"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
new_test_data = [
    ('정성스럽게 만들었는데 왜 이리 대노잼인지 이해가 안 돼', -1), ('돈 주고 갔는데 후회만 가득해서 너무 실망했어', -1),
    ('퍼즐이 너무 어려워서 즐기는 게 아니라 고생하는 느낌이었어', -1), ('이런 걸로 돈 주고 시간 낭비하는 게 너무 아까워', -1),
    ('하고 나니까 정말 즐거운 게 없어서 정말 심심하고 지루했어', -1)
]

new_test_data = pd.DataFrame(new_test_data, columns=['content', 'fun'])
test_data = pd.concat([test_data, new_test_data])

test_data['content'] = test_data['content'].apply(cleaned_content)
test_data['content'] = test_data['content'].apply(kiwi_clean)

In [6]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 959/959 [00:13<00:00, 70.27it/s]


,content,fun,pred
0,스토리 좋다 문제 아쉽다 아직 많이 부족,0,0
1,인 장치 많다 무난 테마,0,0
2,재밌다 바보 열다 안 열다 복기 때 더 재밌다 신묘 테마 버전 걸리다,1,1
3,개인 공간 사용 부분 좋다 따다 두 문제 허 이다 가이드 별로 이다 생각 꽃길 너무...,0,0
4,테마 기준 걸리다,0,0
...,...,...,...
0,정성 만들다 왜 이리 대노잼 이다 이해 안 되다,-1,1
1,돈 주다 가다 후회 가득 너무 실망,-1,1
2,퍼즐 너무 어렵다 즐기다 아니다 고생 느낌 이다,-1,0
3,이런 돈 주다 시간 낭비 너무 아깝다,-1,-1


In [7]:
print('0포함', '\n', classification_report(test_data['fun'], test_data['pred']))
test_data2 = test_data[test_data['fun'] != 0]
print('0 제외', '\n', classification_report(test_data2['fun'], test_data2['pred']))

0포함 
               precision    recall  f1-score   support

          -1       0.12      0.29      0.17         7
           0       0.94      0.92      0.93       775
           1       0.71      0.76      0.73       177

    accuracy                           0.88       959
   macro avg       0.59      0.65      0.61       959
weighted avg       0.89      0.88      0.89       959

0 제외 
               precision    recall  f1-score   support

          -1       1.00      0.29      0.44         7
           0       0.00      0.00      0.00         0
           1       0.97      0.76      0.85       177

    accuracy                           0.74       184
   macro avg       0.66      0.35      0.43       184
weighted avg       0.97      0.74      0.84       184



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [8]:
test = ['재밌어요!', '존잼존잼', '무섭긴 한데 재밌었음!!', 
'노잼', '진짜 돈 아까움. 노잼 그 자체', '재미 하나도 없어요', '스토리도 없고 재미도 없고 인테리어도 없고',
'대존잼', '진짜진짜 재밌었어요', '스토리 그냥 저냥']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent} → 평가: {pred}')
    print("-"*30, '\n')

문장: 재밌어요! → 평가: 0
------------------------------ 

문장: 존잼존잼 → 평가: 0
------------------------------ 

문장: 무섭긴 한데 재밌었음!! → 평가: 1
------------------------------ 

문장: 노잼 → 평가: 0
------------------------------ 

문장: 진짜 돈 아까움. 노잼 그 자체 → 평가: 2
------------------------------ 

문장: 재미 하나도 없어요 → 평가: 0
------------------------------ 

문장: 스토리도 없고 재미도 없고 인테리어도 없고 → 평가: 0
------------------------------ 

문장: 대존잼 → 평가: 1
------------------------------ 

문장: 진짜진짜 재밌었어요 → 평가: 2
------------------------------ 

문장: 스토리 그냥 저냥 → 평가: 1
------------------------------ 

